<a href="https://colab.research.google.com/github/TheFourStooges/TheFourStooges.github.io/blob/main/test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#garbage collector
import gc
gc.enable() #enabling collection

from zipfile import ZipFile


In [2]:
PATH = '/kaggle/input/instacart-market-basket-analysis'

In [3]:
zip_file1 = ZipFile('/kaggle/input/instacart-market-basket-analysis/order_products__train.csv.zip')
zip_file2 = ZipFile('/kaggle/input/instacart-market-basket-analysis/order_products__prior.csv.zip')
zip_file3 = ZipFile('/kaggle/input/instacart-market-basket-analysis/departments.csv.zip')
zip_file4 = ZipFile('/kaggle/input/instacart-market-basket-analysis/sample_submission.csv.zip')
zip_file5 = ZipFile('/kaggle/input/instacart-market-basket-analysis/orders.csv.zip')
zip_file6 = ZipFile('/kaggle/input/instacart-market-basket-analysis/products.csv.zip')
zip_file7 = ZipFile('/kaggle/input/instacart-market-basket-analysis/aisles.csv.zip')

In [4]:
print(zip_file1.namelist())
print(zip_file2.namelist())
print(zip_file3.namelist())
print(zip_file4.namelist())
print(zip_file5.namelist())
print(zip_file6.namelist())
print(zip_file7.namelist())

['order_products__train.csv', '__MACOSX/', '__MACOSX/._order_products__train.csv']
['order_products__prior.csv', '__MACOSX/', '__MACOSX/._order_products__prior.csv']
['departments.csv', '__MACOSX/', '__MACOSX/._departments.csv']
['sample_submission.csv']
['orders.csv', '__MACOSX/', '__MACOSX/._orders.csv']
['products.csv', '__MACOSX/', '__MACOSX/._products.csv']
['aisles.csv', '__MACOSX/', '__MACOSX/._aisles.csv']


In [5]:
# read csv file -> Dataframe
order_products_train = pd.read_csv(zip_file1.open('order_products__train.csv'))
order_products_prior = pd.read_csv(zip_file2.open('order_products__prior.csv'))
departments = pd.read_csv(zip_file3.open('departments.csv'))
sample_submission = pd.read_csv(zip_file4.open('sample_submission.csv'))
orders = pd.read_csv(zip_file5.open('orders.csv'))
products = pd.read_csv(zip_file6.open('products.csv'), usecols=['product_id', 'aisle_id', 'department_id'])
orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2})
orders.days_since_prior_order = orders.days_since_prior_order.fillna(30)
# aisles = pd.read_csv(zip_file7.open('aisles.csv'))

In [6]:
#merging orders and prior datasets
prior_orders = pd.merge(orders, order_products_prior, on='order_id', how='inner')
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,0,1,2,8,30.0,196,1,0
1,2539329,1,0,1,2,8,30.0,14084,2,0
2,2539329,1,0,1,2,8,30.0,12427,3,0
3,2539329,1,0,1,2,8,30.0,26088,4,0
4,2539329,1,0,1,2,8,30.0,26405,5,0


In [7]:
#deleting prior dataset
del order_products_prior
gc.collect()

23

In [8]:
#number of orders placed by each user.
users = prior_orders.groupby(by='user_id')['order_number'].aggregate('max').to_frame('u_num_of_orders').reset_index()
# #converting the datatype to int.
# users.u_num_of_orders = users.u_num_of_orders.astype(np.uint8)
users.head()

,user_id,u_num_of_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [9]:
#average products in orders placed by each users.

#1. First getting the total number of products in each order.
total_prd_per_order = prior_orders.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()

#2. Getting the average products purchased by each user
avg_products = total_prd_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('u_avg_prd').reset_index()
avg_products.head()

#deleting the total_prd_per_order dataframe
del [total_prd_per_order]
gc.collect()

avg_products.head()

,user_id,u_avg_prd
0,1,5.900000
1,2,13.928571
2,3,7.333333
3,4,3.600000
4,5,9.250000


In [10]:
#dow the user has ordered most.
#importing the scipy's stats model
# from scipy import stats


# #execution will take approx 45sec.
# dow = prior_orders.groupby(by=['user_id'])['order_dow'].aggregate(lambda x : stats.mode(x)[0]).to_frame('dow_u_most_orders')
# #resetting the index
# dow = dow.reset_index()
# dow.head()
# dow = prior_orders.groupby('user_id')['order_dow'].agg(pd.Series.mode).to_frame('dow_u_most_orders')
# dow = dow.reset_index()
# dow.head()

from scipy import stats

dow = prior_orders.groupby('user_id')['order_dow'].agg(lambda x: stats.mode(x)[0][0]).to_frame('dow_u_most_orders')
dow = dow.reset_index()
dow.head()




,user_id,dow_u_most_orders
0,1,4
1,2,2
2,3,0
3,4,4
4,5,3


In [11]:
#hour of day the user has ordered most.

#execution will take approx 45sec.
hod = prior_orders.groupby('user_id')['order_hour_of_day'].agg(lambda x: stats.mode(x)[0][0]).to_frame('hod_u_most_orders')
hod = hod.reset_index()
hod.head()

,user_id,hod_u_most_orders
0,1,7
1,2,9
2,3,16
3,4,15
4,5,18


In [12]:
#reorder ratio of user.
reorder_u = prior_orders.groupby('user_id')['reordered'].aggregate('mean').to_frame('u_reorder_ratio').reset_index()
#changing the dtype.
reorder_u['u_reorder_ratio'] = reorder_u['u_reorder_ratio'].astype(np.float16)
reorder_u.head()

,user_id,u_reorder_ratio
0,1,0.694824
1,2,0.476807
2,3,0.625000
3,4,0.055542
4,5,0.378418


In [13]:
#filling the NAN values with 0.
prior_orders.days_since_prior_order.fillna(0, inplace=True)


In [14]:
#average days between orders.
avg_days = prior_orders.groupby(by='user_id')['days_since_prior_order'].aggregate('mean').to_frame('average_days_between_orders')
#resetting index
avg_days = avg_days.reset_index()
avg_days.head()

,user_id,average_days_between_orders
0,1,21.084746
1,2,16.902564
2,3,13.590909
3,4,18.611111
4,5,19.108108


In [15]:
#total items bought.
total_item = prior_orders.groupby(by='user_id').size().to_frame('u_total_items_bought').astype(np.int16)
total_item.head()

,u_total_items_bought
user_id,
1,59
2,195
3,88
4,18
5,37


In [16]:
#merging users df and avg_prd
users = users.merge(avg_products, on='user_id', how='left')
#merging users df with dow
users = users.merge(dow, on='user_id', how='left')
#merging users df with hod
users = users.merge(hod, on='user_id', how='left')
#merging users df with reorder_u
users = users.merge(reorder_u, on='user_id', how='left')
#merging users df with avg_days
users = users.merge(avg_days, on='user_id', how='left')
#merging total_item df with reorder_u
users = users.merge(total_item, on='user_id', how='left')

users.head()

,user_id,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought
0,1,10,5.900000,4,7,0.694824,21.084746,59
1,2,14,13.928571,2,9,0.476807,16.902564,195
2,3,12,7.333333,0,16,0.625000,13.590909,88
3,4,5,3.600000,4,15,0.055542,18.611111,18
4,5,4,9.250000,3,18,0.378418,19.108108,37


In [17]:
#number of times purchased.
prd = prior_orders.groupby(by='product_id')['order_id'].aggregate('count').to_frame('p_num_of_times').reset_index()
# prd['p_num_of_times'] = prd['p_num_of_times'].astype(np.uint16)
prd.head()

,product_id,p_num_of_times
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [18]:
#reordered ratio for each product
reorder_p = prior_orders.groupby(by='product_id')['reordered'].aggregate('mean').to_frame('p_reorder_ratio').reset_index()
# #changing dtype
# reorder_p['p_reorder_ratio'] = reorder_p['p_reorder_ratio'].astype(np.float16)
reorder_p.head()

,product_id,p_reorder_ratio
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [19]:
#add to cart for each product.
add_to_cart = prior_orders.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').to_frame('p_avg_cart_position').reset_index()
# #changing the dtype
# add_to_cart['p_avg_cart_position'] = add_to_cart['p_avg_cart_position'].astype(np.float16)
add_to_cart.head()

,product_id,p_avg_cart_position
0,1,5.801836
1,2,9.888889
2,3,6.415162
3,4,9.507599
4,5,6.466667


In [20]:
#merging reorder_p with prd.
prd = prd.merge(reorder_p, on='product_id', how='left')

#merging add_to_cart with prd.
prd = prd.merge(add_to_cart, on='product_id', how='left')

#deleting unwanted df.
del [reorder_p, add_to_cart]
gc.collect()

23

In [21]:
prd.head()

,product_id,p_num_of_times,p_reorder_ratio,p_avg_cart_position
0,1,1852,0.613391,5.801836
1,2,90,0.133333,9.888889
2,3,277,0.732852,6.415162
3,4,329,0.446809,9.507599
4,5,15,0.600000,6.466667


In [22]:
#times a user have bough a product.
uxp = prior_orders.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('uxp_times_bought')
#resetting index
uxp = uxp.reset_index()
# #changing the dtype.
# uxp['uxp_times_bought'] = uxp['uxp_times_bought'].astype(np.uint8)
uxp.head()

,user_id,product_id,uxp_times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [23]:
#times a user have bough a product.
times = prior_orders.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('times_bought')
#resetting index
times = times.reset_index()
# #changing the dtype.
# times['times_bought'] = times['times_bought'].astype(np.uint8)
times.head()

,user_id,product_id,times_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [24]:
#Total orders
total_orders = prior_orders.groupby('user_id')['order_number'].max().to_frame('total_orders').reset_index()
total_orders.head()

,user_id,total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [25]:
#Finding when the user has bought a product the first time.
first_order_num = prior_orders.groupby(by=['user_id', 'product_id'])['order_number'].aggregate('min').to_frame('first_order_num')
#resetting the index
first_order_num = first_order_num.reset_index()
first_order_num.head()

,user_id,product_id,first_order_num
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [26]:
#merging both the dataframes
span = pd.merge(total_orders, first_order_num, on='user_id', how='right')
span.head()

,user_id,total_orders,product_id,first_order_num
0,1,10,196,1
1,1,10,10258,2
2,1,10,10326,5
3,1,10,12427,1
4,1,10,13032,2


In [27]:
#Calculating the order range.
# The +1 includes in the difference the first order were the product has been purchased
span['Order_Range_D'] = span.total_orders - span.first_order_num + 1
span.head()

,user_id,total_orders,product_id,first_order_num,Order_Range_D
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [28]:
#merging times df with the span
uxp_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

,user_id,product_id,times_bought,total_orders,first_order_num,Order_Range_D
0,1,196,10,10,1,10
1,1,10258,9,10,2,9
2,1,10326,1,10,5,6
3,1,12427,10,10,1,10
4,1,13032,3,10,2,9


In [29]:
#calculating the ratio.
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio.times_bought / uxp_ratio.Order_Range_D
uxp_ratio.head()

,user_id,product_id,times_bought,total_orders,first_order_num,Order_Range_D,uxp_reorder_ratio
0,1,196,10,10,1,10,1.000000
1,1,10258,9,10,2,9,1.000000
2,1,10326,1,10,5,6,0.166667
3,1,12427,10,10,1,10,1.000000
4,1,13032,3,10,2,9,0.333333


In [30]:
#dropping all the unwanted columns.
uxp_ratio.drop(['times_bought', 'total_orders', 'first_order_num', 'Order_Range_D'], axis=1, inplace=True)
uxp_ratio.head()

,user_id,product_id,uxp_reorder_ratio
0,1,196,1.000000
1,1,10258,1.000000
2,1,10326,0.166667
3,1,12427,1.000000
4,1,13032,0.333333


In [31]:
#deleting all the unwanted df.
del [times, span, first_order_num, total_orders]
gc.collect()

87

In [32]:
#merging uxp_ratio with uxp.
uxp = uxp.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')
#deleting uxp_ratio
del uxp_ratio
#calling garbage collector.
gc.collect()

23

In [33]:
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio
0,1,196,10,1.000000
1,1,10258,9,1.000000
2,1,10326,1,0.166667
3,1,12427,10,1.000000
4,1,13032,3,0.333333


In [34]:
#Reversing the order number for each product.
prior_orders['order_number_back'] = prior_orders.groupby(by=['user_id'])['order_number'].transform(max) - prior_orders.order_number + 1
prior_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
0,2539329,1,0,1,2,8,30.0,196,1,0,10
1,2539329,1,0,1,2,8,30.0,14084,2,0,10
2,2539329,1,0,1,2,8,30.0,12427,3,0,10
3,2539329,1,0,1,2,8,30.0,26088,4,0,10
4,2539329,1,0,1,2,8,30.0,26405,5,0,10


In [35]:
#keeping only the first 5 orders from the order_number_back.
temp = prior_orders.loc[prior_orders.order_number_back <= 5]
temp.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back
29,3367565,1,0,6,2,7,19.0,196,1,1,5
30,3367565,1,0,6,2,7,19.0,12427,2,1,5
31,3367565,1,0,6,2,7,19.0,10258,3,1,5
32,3367565,1,0,6,2,7,19.0,25133,4,1,5
33,550135,1,0,7,1,9,20.0,196,1,1,4


In [36]:
#product bought by users in the last_five orders.
last_five = temp.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('uxp_last_five').reset_index()
last_five.head()

,user_id,product_id,uxp_last_five
0,1,196,5
1,1,10258,5
2,1,12427,5
3,1,13032,2
4,1,25133,5


In [37]:
#ratio of the products bought in the last_five orders.
last_five['uxp_ratio_last_five'] = last_five.uxp_last_five / 5.0
# #changing the dtype.
# last_five['uxp_ratio_last_five'] = last_five['uxp_ratio_last_five'].astype(np.float16)
last_five.head()

,user_id,product_id,uxp_last_five,uxp_ratio_last_five
0,1,196,5,1.0
1,1,10258,5,1.0
2,1,12427,5,1.0
3,1,13032,2,0.4
4,1,25133,5,1.0


In [38]:
#merging this feature with uxp df.
uxp = uxp.merge(last_five, on=['user_id', 'product_id'], how='left')

del [last_five, temp]
gc.collect()
uxp.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five
0,1,196,10,1.000000,5.0,1.0
1,1,10258,9,1.000000,5.0,1.0
2,1,10326,1,0.166667,NaN,NaN
3,1,12427,10,1.000000,5.0,1.0
4,1,13032,3,0.333333,2.0,0.4


In [39]:
#filling the NAN values with 0.
uxp.fillna(0, inplace=True)
uxp.head(10)

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five
0,1,196,10,1.000000,5.0,1.0
1,1,10258,9,1.000000,5.0,1.0
2,1,10326,1,0.166667,0.0,0.0
3,1,12427,10,1.000000,5.0,1.0
4,1,13032,3,0.333333,2.0,0.4
5,1,13176,2,0.222222,0.0,0.0
6,1,14084,1,0.100000,0.0,0.0
7,1,17122,1,0.166667,0.0,0.0
8,1,25133,8,1.000000,5.0,1.0
9,1,26088,2,0.200000,0.0,0.0


In [40]:
uxp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int64  
 1   product_id           int64  
 2   uxp_times_bought     int64  
 3   uxp_reorder_ratio    float64
 4   uxp_last_five        float64
 5   uxp_ratio_last_five  float64
dtypes: float64(3), int64(3)
memory usage: 710.7 MB


In [41]:
# #Merge uxp features with the user features
# #Store the results on a new DataFrame
data = uxp.merge(users, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought
0,1,196,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59
1,1,10258,9,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59
2,1,10326,1,0.166667,0.0,0.0,10,5.9,4,7,0.694824,21.084746,59
3,1,12427,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59
4,1,13032,3,0.333333,2.0,0.4,10,5.9,4,7,0.694824,21.084746,59


In [42]:
#Merging prd features with data.
data = data.merge(prd, on='product_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought,p_num_of_times,p_reorder_ratio,p_avg_cart_position
0,1,196,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,35791,0.776480,3.721774
1,1,10258,9,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,1946,0.713772,4.277492
2,1,10326,1,0.166667,0.0,0.0,10,5.9,4,7,0.694824,21.084746,59,5526,0.652009,4.191097
3,1,12427,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,6476,0.740735,4.760037
4,1,13032,3,0.333333,2.0,0.4,10,5.9,4,7,0.694824,21.084746,59,3751,0.657158,5.622767


In [43]:
#deleting unwanted df.
del [users, prd, uxp]
gc.collect()

23

In [44]:
#shape of the dataset.
data.shape

(13307953, 16)

In [45]:
#keeping only the train and test set from the orders df.
orders_future = orders.loc[((orders.eval_set == 1) | (orders.eval_set == 2)), ['user_id', 'eval_set', 'order_id']]
orders_future.head()

,user_id,eval_set,order_id
10,1,1,1187899
25,2,1,1492625
38,3,2,2774568
44,4,2,329954
49,5,1,2196797


In [46]:
#merging the orders_future with data.
data = data.merge(orders_future, on='user_id', how='left')
data.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought,p_num_of_times,p_reorder_ratio,p_avg_cart_position,eval_set,order_id
0,1,196,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,35791,0.776480,3.721774,1,1187899
1,1,10258,9,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,1946,0.713772,4.277492,1,1187899
2,1,10326,1,0.166667,0.0,0.0,10,5.9,4,7,0.694824,21.084746,59,5526,0.652009,4.191097,1,1187899
3,1,12427,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,6476,0.740735,4.760037,1,1187899
4,1,13032,3,0.333333,2.0,0.4,10,5.9,4,7,0.694824,21.084746,59,3751,0.657158,5.622767,1,1187899


In [47]:
del orders_future
gc.collect()

105

In [48]:
#Preparing training data set.
data_train = data[data.eval_set == 1]
data_train.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought,p_num_of_times,p_reorder_ratio,p_avg_cart_position,eval_set,order_id
0,1,196,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,35791,0.776480,3.721774,1,1187899
1,1,10258,9,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,1946,0.713772,4.277492,1,1187899
2,1,10326,1,0.166667,0.0,0.0,10,5.9,4,7,0.694824,21.084746,59,5526,0.652009,4.191097,1,1187899
3,1,12427,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,6476,0.740735,4.760037,1,1187899
4,1,13032,3,0.333333,2.0,0.4,10,5.9,4,7,0.694824,21.084746,59,3751,0.657158,5.622767,1,1187899


In [49]:
#merging the information contained in the order_products__train.csv into data_train.
data_train = data_train.merge(order_products_train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
data_train.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought,p_num_of_times,p_reorder_ratio,p_avg_cart_position,eval_set,order_id,reordered
0,1,196,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,35791,0.776480,3.721774,1,1187899,1.0
1,1,10258,9,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,1946,0.713772,4.277492,1,1187899,1.0
2,1,10326,1,0.166667,0.0,0.0,10,5.9,4,7,0.694824,21.084746,59,5526,0.652009,4.191097,1,1187899,NaN
3,1,12427,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,6476,0.740735,4.760037,1,1187899,NaN
4,1,13032,3,0.333333,2.0,0.4,10,5.9,4,7,0.694824,21.084746,59,3751,0.657158,5.622767,1,1187899,1.0


In [50]:
#filling the NAN values in the reordered
data_train.reordered.fillna(0, inplace=True)

In [51]:
# #setting user_id and product_id as index.
# data_train = data_train.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_train.drop(['eval_set', 'order_id'], axis=1, inplace=True)

In [52]:
#head()
data_train.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought,p_num_of_times,p_reorder_ratio,p_avg_cart_position,reordered
0,1,196,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,35791,0.776480,3.721774,1.0
1,1,10258,9,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,1946,0.713772,4.277492,1.0
2,1,10326,1,0.166667,0.0,0.0,10,5.9,4,7,0.694824,21.084746,59,5526,0.652009,4.191097,0.0
3,1,12427,10,1.000000,5.0,1.0,10,5.9,4,7,0.694824,21.084746,59,6476,0.740735,4.760037,0.0
4,1,13032,3,0.333333,2.0,0.4,10,5.9,4,7,0.694824,21.084746,59,3751,0.657158,5.622767,1.0


In [53]:
#Preparing the test dataset.
data_test = data[data.eval_set == 2]
data_test.head()

,user_id,product_id,uxp_times_bought,uxp_reorder_ratio,uxp_last_five,uxp_ratio_last_five,u_num_of_orders,u_avg_prd,dow_u_most_orders,hod_u_most_orders,u_reorder_ratio,average_days_between_orders,u_total_items_bought,p_num_of_times,p_reorder_ratio,p_avg_cart_position,eval_set,order_id
120,3,248,1,0.090909,0.0,0.0,12,7.333333,0,16,0.625,13.590909,88,6371,0.400251,10.620782,2,2774568
121,3,1005,1,0.333333,1.0,0.2,12,7.333333,0,16,0.625,13.590909,88,463,0.440605,9.498920,2,2774568
122,3,1819,3,0.333333,0.0,0.0,12,7.333333,0,16,0.625,13.590909,88,2424,0.492162,9.287541,2,2774568
123,3,7503,1,0.100000,0.0,0.0,12,7.333333,0,16,0.625,13.590909,88,12474,0.553551,9.547379,2,2774568
124,3,8021,1,0.090909,0.0,0.0,12,7.333333,0,16,0.625,13.590909,88,27864,0.591157,8.822854,2,2774568


In [54]:
# #setting user_id and product_id as index.
# data_test = data_test.set_index(['user_id', 'product_id'])

#deleting eval_set, order_id as they are not needed for training.
data_test.drop(['eval_set', 'order_id'], axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [55]:
#shape of train and test.
data_train.shape, data_test.shape

((8474661, 17), (4833292, 16))

In [56]:
# #resetting index
# data_train.reset_index()
# data_test.reset_index()

#adding aisle and department data
# products = pd.read_csv('../content/drive/MyDrive/Colab Notebooks/' + 'products.csv', usecols=['product_id', 'aisle_id', 'department_id'])

#merging product data into data_train and data_test.
data_train = data_train.merge(products, on='product_id', how='left')
data_test = data_test.merge(products, on='product_id', how='left')

#setting the index again
data_train = data_train.set_index(['user_id', 'product_id'])
data_test = data_test.set_index(['user_id', 'product_id'])

In [57]:
#mean encoding categorical variables.
columns_mean = ['aisle_id', 'department_id']
for col in columns_mean:
        mean = data_train.groupby(col).reordered.mean()
        data_train[col] = data_train[col].map(mean)
        data_test[col] = data_test[col].map(mean)

In [60]:
data_train.to_csv('data_train.csv', index=False, header=True)

In [59]:
data_test

uxp_times_bought  uxp_reorder_ratio  uxp_last_five  \
user_id product_id                                                       
3       248                        1           0.090909            0.0   
        1005                       1           0.333333            1.0   
        1819                       3           0.333333            0.0   
        7503                       1           0.100000            0.0   
        8021                       1           0.090909            0.0   
...                              ...                ...            ...   
206208  48364                      1           0.021277            0.0   
        48865                      1           0.125000            0.0   
        49247                      1           0.022727            0.0   
        49385                      1           0.043478            0.0   
        49621                      1           0.052632            0.0   

                    uxp_ratio_last_five  u_num_of_orders  u_avg_prd  \
user_id product_id                                                    
3       248                         0.0               12   7.333333   
        1005                        0.2               12   7.333333   
        1819                        0.0               12   7.333333   
        7503                        0.0               12   7.333333   
        8021                        0.0               12   7.333333   
...                                 ...              ...        ...   
206208  48364                       0.0               49  13.816327   
        48865                       0.0               49  13.816327   
        49247                       0.0               49  13.816327   
        49385                       0.0               49  13.816327   
        49621                       0.0               49  13.816327   

                    dow_u_most_orders  hod_u_most_orders  u_reorder_ratio  \
user_id product_id                                                          
3       248                         0                 16          0.62500   
        1005                        0                 16          0.62500   
        1819                        0                 16          0.62500   
        7503                        0                 16          0.62500   
        8021                        0                 16          0.62500   
...                               ...                ...              ...   
206208  48364                       2                 15          0.70752   
        48865                       2                 15          0.70752   
        49247                       2                 15          0.70752   
        49385                       2                 15          0.70752   
        49621                       2                 15          0.70752   

                    average_days_between_orders  u_total_items_bought  \
user_id product_id                                                      
3       248                           13.590909                    88   
        1005                          13.590909                    88   
        1819                          13.590909                    88   
        7503                          13.590909                    88   
        8021                          13.590909                    88   
...                                         ...                   ...   
206208  48364                          7.841950                   677   
        48865                          7.841950                   677   
        49247                          7.841950                   677   
        49385                          7.841950                   677   
        49621                          7.841950                   677   

                    p_num_of_times  p_reorder_ratio  p_avg_cart_position  \
user_id product_id                                                         
3       248                  